In [20]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from scipy.interpolate.interpolate import interp1d
import plotly.graph_objects as go

In [21]:
class DataHandler:
    def __init__(self, spinner=False, blades=3):
        self.spinner = 's' if spinner else ''
        self.blades = blades
        self.load_data()

    def filename(self):
        return 'clark_{}{}_eff.csv'.format(self.blades, self.spinner)
    
    def load_data(self):
        data = pd.read_csv('data\\large_angles\\' + self.filename(), engine='python')
        data = data.melt(id_vars = ['x'], var_name = 'y', value_name = 'z').dropna()
        data.y = data.y.astype(float)
        self.data = data.reset_index(drop=True)
        
    def save_data(self):
        df_pivot = self.curves.pivot(index='y', columns='x', values='z')
        df_pivot = df_pivot.iloc[::-1].fillna(value=0, limit=2).iloc[::-1]
        df = pd.melt(df_pivot.reset_index(), id_vars='y', value_vars=df_pivot.columns, value_name = 'z')
        self.curves = df[['x', 'y', 'z']].dropna().reset_index(drop=True).sort_values(by=['x', 'y'])
        df_pivot.to_csv('data\\mesh\\pivot\\' + self.filename(), index=False)
        self.curves.to_csv('data\\mesh\\' + self.filename(), index=False)
              
    def appendTo(self, df, x, y, z):
        return df.append(pd.DataFrame({'x': x, 'y': y, 'z': z}), ignore_index=True)
        
        

In [22]:
class Propeller(DataHandler):
    
    def plot_data(self, df, color, name, size=3):
        return go.Scatter3d(
            x=df.x, y=df.y, z=df.z,
            mode='markers',
            marker=dict(size=size, color=color, opacity=0.9),
            name=name)
    
    def layout(self):
        camera = dict(eye=dict(x=1.3, y=-1.3, z=0.5),
                      center=dict(y=-0.1, z=-0.15))
        scene = dict(
            xaxis = dict(title='J'),
            yaxis = dict(title='Angle'),
            zaxis = dict(title='Eff')
        )
        legend = dict(font = {'size': 12}, itemsizing = 'trace')
        return go.Layout(margin=dict(l=0, r=0, b=0, t=0), 
                         width=1024, height=768, 
                         scene=scene,
                         scene_camera=camera,
                         legend_title_text='Data',
                         legend=legend)
    
    def draw(self, y_max=60):
        df = self.data[self.data.y <= y_max]
        fig = go.Figure(layout = self.layout())
        fig.add_trace(self.plot_data(df, 'black', 'NACA', 2))
        if hasattr(self, 'extra') and self.data.min().y == 15:
            fig.add_trace(self.plot_data(self.extra, 'gray', 'New', 4))
        if hasattr(self, 'mesh'):
            df = self.mesh[self.mesh.y <= y_max]
            fig.add_trace(self.plot_data(df, 'red', 'Sides'))
        if hasattr(self, 'top'):
            df = self.top[self.top.y <= y_max]
            fig.add_trace(self.plot_data(df, 'green', 'Top'))
        fig.show()
    
    def drawXZ(self):
        fig = go.Figure(layout = self.layout())
        fig.add_trace(self.plot_data(self.data, 'black', 'NACA'))
        if hasattr(self, 'meshXZ'):
            fig.add_trace(self.plot_data(self.meshXZ, 'gray', 'New'))
        fig.show()
        
    def draw_curves(self, y_max=60, top=True):
        df = self.curves[self.curves.y <= y_max].reset_index(drop=True)
        fig = go.Figure(layout = self.layout())
        fig.add_trace(self.plot_data(df, 'black', 'Siatka', 2))
        if top:
            df = self.top[self.top.y <= y_max].reset_index(drop=True)
            fig.add_trace(self.plot_data(df, 'green', 'Góra'))
        fig.show()

    def draw_surface(self):
        data = self.curves.drop_duplicates(subset=['x', 'y'])
        data = data.pivot(index='y', columns='x', values='z')
        fig = go.Figure(go.Surface(
            z=data.values,
            x=data.columns,
            y=data.index,
            showscale=False,
            colorscale='algae',
            opacity=0.9,
            reversescale=True,
            hovertemplate=
            '<b>J</b>: %{x}<br>' +
            '<b>Angle</b>: %{y}°<br>' +
            '<b>Eff</b>: %{z}<extra></extra>'
        ), layout = self.layout())
        fig.show()
        
    def fit(self):
        fig = go.Figure(layout=go.Layout(margin=dict(l=20, r=20, b=0, t=0), width=240, height=480, showlegend=False))
        points = self.top[self.top.y==10]
        fig.add_trace(go.Scatter(x=points.x, 
                                 y=points.z, 
                                 mode='markers', 
                                 marker=dict(size=5, color='green')))
        data = self.data[self.data.y==10]
        fig.add_trace(go.Scatter(x=data.x, 
                                 y=data.z, 
                                 line=dict(color='black')))
        fig.update_yaxes(range=[-0.02, 0.8])
        fig.update_xaxes(title='J')
        fig.update_yaxes(title='Eff')
        fig.show()
    
    def x_vals(self, begin=0, end=4.8, dense=False):
        vals = np.linspace(0, 5, 5001) if dense else np.linspace(0, 5, 51)
        vals = np.round(vals, 3)
        return vals[ (vals >= begin) & (vals <= end) ]
    
    def y_vals(self, begin=10):
        vals = np.linspace(10, 60, 101)
        vals = np.round(vals, 2)
        return vals[vals >= begin]
    
    def z_vals(self, end=0.8, dense=False):
        vals = np.linspace(0, end, 41) if dense else np.linspace(0, end, 21)
        vals = np.round(vals, 3)
        return vals
        
    def create_data(self):
        points = self.extra[self.extra.y == 10]
        z = interp1d(points.x, points.z, kind='quadratic')
        x = self.x_vals(dense=True, end=points.x.max())
        x = np.append(x, points.x.max())
        self.data = self.appendTo(self.data, x, 10, z(x))
    
    

In [23]:
class Efficiency(Propeller):
        
    def create_curves(self):
        curves = pd.DataFrame()
        bottom_x = self.top[(self.top.z == 0) & (self.top.x > 0)].reset_index(drop=True).x
        x_vals = np.linspace(0, 1, 10, endpoint=False)**0.4*bottom_x[0]
        x_vals = np.append(x_vals, bottom_x)
        for y in prop.y_vals(begin=10):
            inters = self.top[self.top.y == y]
            z = interp1d(inters.x, inters.z, kind='quadratic')
            x = x_vals[x_vals <= inters.x.max()]
            curves = self.appendTo(curves, x, y, z(x))
        self.curves = curves.round({'x': 3, 'z': 4}).drop_duplicates(subset=['x', 'y'])
        
    def densing(self, y_vals):
        """Ratio-based top mesh"""
        mesh = pd.DataFrame()
        z_left = np.linspace(0, 0.99, 10)**0.2 if self.blades == 2 else np.linspace(0, 0.99, 30)**0.7
        z_right = np.linspace(0, 0.99, 5)**0.2 if self.blades == 2 else np.linspace(0, 1, 30)**0.5
        if hasattr(self, 'top'):
            z_left = np.linspace(0, 0.99, 20)**0.7
            z_right = np.linspace(0, 1, 15)**0.4
        for z_rel in z_left:
            points = self.top_intercepts(z_rel, side='left')
            mesh = self.interpolate_y(points, mesh, y_vals)
        for z_rel in z_right:
            points = self.top_intercepts(z_rel, side='right')
            mesh = self.interpolate_y(points, mesh, y_vals)
        self.top = mesh
        
    def interpolate_y(self, points, mesh, y_vals):
        """Get the interpolated x and z values at y_vals"""
        kind = 'linear' if self.blades == 2 and not hasattr(self, 'top') else 'quadratic'
        x = interp1d(points.y, points.x, kind=kind, fill_value='extrapolate')
        z = interp1d(points.y, points.z, kind='quadratic', fill_value='extrapolate')
        return self.appendTo(mesh, x(y_vals), y_vals, z(y_vals))
        
    def interpolate(self, points, mesh, z):
        x = interp1d(points.z, points.x, kind = 'quadratic', fill_value='extrapolate')
        y = interp1d(points.z, points.y, kind = 'quadratic', fill_value='extrapolate')
        mesh = mesh.append({'x': float(x(z)), 
                            'y': float(y(z)), 
                            'z': z}, ignore_index=True)
        return mesh
    
    def top_intercepts(self, relative_z, side):
        '''Points intercepting at the same relative z for every curve'''
        points = pd.DataFrame()
        for y in self.data.y.unique():
            df = self.data[self.data.y == y]
            topX = df.at[df.z.idxmax(), 'x']
            if relative_z == 1:
                points = points.append(df.loc[df.z.idxmax()])
            else:
                curve = df[df.x > topX + 0.03] if side == 'right' else df[df.x < topX - 0.03]
                points = self.interpolate(curve, points, df.z.max() * relative_z)
        return points
        
    def extrapolated_data(self):
        if self.blades == 2:
            '''Top data'''
            self.top.loc[self.top.y==10, 'x'] *= 1.1
            self.extra = self.top
        else:
            '''Polynomial fit approach'''
            points = self.top[self.top.y==10]
            begin = 0.2 if self.blades == 3 else 0.25
            x_vals = np.arange(begin, points.max().x + 0.02, 0.001)
            p, ssr, rank, sv, ct = np.polyfit(points.x, points.z, deg=5, full=True)
            print(round(ssr[0], 5))
            z = np.poly1d(p)
            extra = pd.DataFrame({'x': x_vals, 'y': 10, 'z': z(x_vals)})
            extra = extra.append({'x': 0, 'y': 10, 'z': 0}, ignore_index=True)
            self.extra = extra[extra.z >= 0]
    
    

In [24]:
prop = Efficiency(blades=4)
prop.densing(y_vals=[10])
prop.extrapolated_data()
prop.create_data()
prop.fit()
prop.densing(y_vals=prop.y_vals())
prop.draw(y_max=60)


0.09854


In [25]:
prop.create_curves()
prop.save_data()

In [26]:
prop.draw_curves(y_max=60, top=False)

In [27]:
prop.draw_surface()